In [ ]:
pip install langchain_community tiktoken langchain_openai langchainhub chromadb langchain openai

In [ ]:
pip install bs4 langchain_chroma chromadb pysqlite3-binary 

In [ ]:
import os
import getpass
from openai import AzureOpenAI
os.environ["AZURE_OPENAI_KEY"] = "5985113856904cbba04bf152a1a13f86"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://genaisolution02.openai.azure.com/"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
import chromadb

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    # dimensions: Optional[int] = None, # Can specify dimensions with new text-embedding-3 models
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
)

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding = embeddings)

In [ ]:

# LLM
from langchain.llms import AzureOpenAI
llm = AzureOpenAI(model="gpt-35-turbo",
    #message =[{"role": "system", "content": "You are a helpful assistant."}],              
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
    ,temperature=0.2,)


In [ ]:
from langchain_openai import AzureChatOpenAI

llm1 = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-07-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    temperature=0.2,)

In [ ]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm1
    | StrOutputParser()
)

rag_chain.invoke("What is LLM fine tuning?")

In [ ]:
rag_chain.invoke("what is use of LLM? ")